In [1362]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator, WeekdayLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
%matplotlib gtk

In [1363]:
## Import data frome pickle generated from hotline_daily.py
data = pd.read_pickle('pickle_hotline_0904.pkl')

In [1364]:
## Have a look at the data structure
data.head(3)

,month,year,0,1,2,3,4,5,6,7,...,21,22,23,24,summa,xlday,day,date,week,weekday
tix,11,2016,00:00-00:30,00:30-01:30,01:30-02:30,02:30-03:30,03:30-04:30,04:30-05:30,05:30-06:30,06:30-07:30,...,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00,NaN,42688.0,14,2016-11-14,46,Mon
angekommen,11,2016,0,0,0,0,0,0,1,5,...,0,1,0,0,321,42688.0,14,2016-11-14,46,Mon
verbunden,11,2016,0,0,0,0,0,0,0,4,...,0,0,0,0,221,42688.0,14,2016-11-14,46,Mon


In [1365]:
##### HERE SOME FILTERING CAN TAKE PLACE
#data = data.ix[((data['year']==2017) & (data['week']==9))]
#dataset_title='KW09/17'

#data = data.ix[((data['year']==2017) & (data['month']==1))]
#dataset_title='Nov 2016 - Apr 2017'

data = data.ix[((data['year']==2017) & (data['month']==3))]
dataset_title='Mar 2017'

# without any filtering:
#dataset_title='Nov 2016 - now'

In [1366]:
## separate the sums of calls, verbundene and lost into different dataframes; this will avoid tuple indexing
calls_d=data[['date','summa','weekday','day']].loc['angekommen'].set_index("date")
verbu_d=data[['date','summa','weekday','day']].loc['verbunden'].set_index("date")
verlo_d=data[['date','summa','weekday','day']].loc['verloren'].set_index("date")

## have a look at one of the new dataframes
print calls_d.index
calls_d.head(2)

Index([2017-03-01, 2017-03-02, 2017-03-03, 2017-03-04, 2017-03-05, 2017-03-06,
       2017-03-07, 2017-03-08, 2017-03-09, 2017-03-10, 2017-03-11, 2017-03-12,
       2017-03-13, 2017-03-14, 2017-03-15, 2017-03-16, 2017-03-17, 2017-03-18,
       2017-03-19, 2017-03-20, 2017-03-21, 2017-03-22, 2017-03-23, 2017-03-24,
       2017-03-25, 2017-03-26, 2017-03-27, 2017-03-28, 2017-03-29, 2017-03-30,
       2017-03-31],
      dtype='object', name=u'date')


,summa,weekday,day
date,,,
2017-03-01,38,Wed,1
2017-03-02,76,Thu,2


In [1367]:
## merge dataframes again into one, containing the sums and indexed by date.
## this seems kind of redundant, given the frames above. guess it should be refactored; but works for now

dayf = pd.DataFrame(index=calls_d.index)
dayf[['angekommen','weekday','day']]=calls_d[['summa','weekday','day']]
dayf['verbunden']=verbu_d['summa']
dayf['verloren']=verlo_d['summa']
dayf=dayf[['weekday','day','angekommen','verbunden','verloren']]

## take a glance at the new dataframe
dayf.head(2)

,weekday,day,angekommen,verbunden,verloren
date,,,,,
2017-03-01,Wed,1,38,26,12
2017-03-02,Thu,2,76,59,17


In [1368]:
## group by weekday for an analysis of the average week
def av_week(d_frame):
    avweek = d_frame[['weekday','angekommen','verbunden','verloren']].groupby(['weekday']).sum()
    day_counter=dayf['weekday'].value_counts().reindex(list(calendar.day_abbr))
    avweek = avweek.reindex(list(calendar.day_abbr)).reset_index()
    avweek['daycount']=day_counter.values
    avweek['av_angekommen']=np.round(avweek['angekommen']/avweek['daycount'],decimals=2)
    return avweek

In [1369]:
awk=av_week(dayf)
awk

,weekday,angekommen,verbunden,verloren,daycount,av_angekommen
0,Mon,202.0,161.0,41.0,4,50.5
1,Tue,178.0,164.0,14.0,4,44.5
2,Wed,225.0,187.0,38.0,5,45.0
3,Thu,313.0,276.0,37.0,5,62.6
4,Fri,244.0,227.0,17.0,5,48.8
5,Sat,96.0,86.0,10.0,4,24.0
6,Sun,14.0,9.0,5.0,4,3.5


In [1370]:
## this will generate the ticks on the x-Axis of the plot
## I don't fully understand this, though. more reading is due
xtick_locator = WeekdayLocator(byweekday=MO)
xtick_formatter = DateFormatter('%W/%y')

#print(xtick_locator)
#print(xtick_formatter)



In [1371]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_daily_abs(d_frame,title):
    
    dayf = d_frame # just for the sake of not having to rewrite it all
    
    fig = plt.figure(figsize=(18,6))
    fig.add_subplot(111)
    plt.ylabel('Calls')
    plt.xlabel('Kalenderwoche')
    plt.margins(0.01)
    plt.title(title)

    # "Initialize" Axes to configure them later
    ax = plt.axes()
    ax.set_axisbelow(True)

    # Y-Axis: major ticks every 50, minor ticks every 10                                    
    minor_ticks = np.arange(0, dayf.angekommen.max(), 10)
    major_ticks = np.arange(0, dayf.angekommen.max(), 50)
    ax.yaxis.grid(True)
    ax.set_yticks(minor_ticks, minor=True)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.xaxis.set_major_locator(xtick_locator)
    ax.xaxis.set_major_formatter(xtick_formatter)

    # Bars for each sum and day
    plt.bar(dayf.index,dayf.angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    plt.bar(dayf.index,dayf.verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
    plt.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

    # Markers for Mondays and Months
    plt.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize='4', linestyle='none', label='Montage')
    plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

    # Legend and Layout formatting after the plotting has taken place
    plt.legend()
    plt.tight_layout()


In [1372]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_weekly_abs(d_frame,title):
    xtick_locator = WeekdayLocator(byweekday=[MO,TU,WE,TH,FR,SA,SU])
    xtick_formatter = DateFormatter('%a,%d.%m.%y')
    dayf = d_frame # just for the sake of not having to rewrite it all
    
    fig = plt.figure(figsize=(5,4))
    fig.add_subplot(111)
    plt.ylabel('Calls')
    plt.xlabel('Kalenderwoche')
    plt.margins(0.01)
    plt.title(title)
    plt.tick_params(axis='both', labelsize=6)

    # "Initialize" Axes to configure them later
    ax = plt.axes()
    ax.set_axisbelow(True)

    # Y-Axis: major ticks every 50, minor ticks every 10                                    
    minor_ticks = np.arange(0, dayf.angekommen.max(), 2)
    major_ticks = np.arange(0, dayf.angekommen.max(), 10)
    ax.yaxis.grid(True)
    ax.set_yticks(minor_ticks, minor=True)
    ax.set_yticks(major_ticks)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.xaxis.set_major_locator(xtick_locator)
    ax.xaxis.set_major_formatter(xtick_formatter)

    # Bars for each sum and day
    plt.bar(dayf.index,dayf.angekommen,width=0.6, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    plt.bar(dayf.index,dayf.verbunden,width=0.4, alpha=1, color='#00B9FF', label='verbunden', align='center')
    plt.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

    # Markers for Mondays and Months
    #plt.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize='4', linestyle='none', label='Montage')
    #plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

    # Legend and Layout formatting after the plotting has taken place
    plt.legend(loc=0,prop={'size':6})
    plt.tight_layout()


In [1373]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_monthly_abs(d_frame,title):
    xtick_locator = WeekdayLocator(byweekday=[MO,TU,WE,TH,FR,SA,SU])
    xtick_formatter = DateFormatter('%d.%m.')
    dayf = d_frame # just for the sake of not having to rewrite it all
    
    fig = plt.figure(figsize=(10,4))
    fig.add_subplot(111)
    plt.ylabel('Calls')
    plt.xlabel('Tage / Monat')
    plt.margins(0.01)
    plt.title(title)
    plt.tick_params(axis='both', labelsize=7)

    # "Initialize" Axes to configure them later
    ax = plt.axes()
    ax.set_axisbelow(True)

    # Y-Axis: major ticks every 50, minor ticks every 10                                    
    minor_ticks = np.arange(0, dayf.angekommen.max(), 2)
    major_ticks = np.arange(0, dayf.angekommen.max(), 10)
    ax.yaxis.grid(True)
    ax.set_yticks(minor_ticks, minor=True)
    ax.set_yticks(major_ticks)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.xaxis.set_major_locator(xtick_locator)
    ax.xaxis.set_major_formatter(xtick_formatter)

    # Bars for each sum and day
    plt.bar(dayf.index,dayf.angekommen,width=0.6, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    plt.bar(dayf.index,dayf.verbunden,width=0.4, alpha=1, color='#00B9FF', label='verbunden', align='center')
    plt.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

    # Markers for Mondays and Months
    plt.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize='4', linestyle='none', label='Montage')
    #plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

    # Legend and Layout formatting after the plotting has taken place
    plt.legend(loc=0,prop={'size':6})
    plt.tight_layout()

In [1374]:
#plot_weekly_abs(dayf,dataset_title)
#plot_monthly_abs(dayf,dataset_title)

In [1375]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_average_week(d_frame,title):
    
    dayf = d_frame # just for the sake of not having to rewrite it all
    
    fig = plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)
    ax.set_ylabel('Gesamtanzahl Calls')
    
    #ax.margins(0.01)
    

    # "Initialize" Axes to configure them later
    # ax = plt.axes()
    

    # Y-Axis: set minor ticks to every 10 of major                                 
    ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.yaxis.grid(True)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.set_axisbelow(True)
    #ax.set_yticks(minor_ticks, minor=True)
    #ax.set_yticks(major_ticks)
    
    #ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    #ax.xaxis.set_major_locator(xtick_locator)
    #ax.xaxis.set_major_formatter(xtick_formatter)
    ax.set_xlabel('Wochentag')
    ax.set_xticks(np.arange(0,7,1))
    ax.set_xticklabels(dayf.weekday)
    
    
    # Another x-Axis for average number
    ax2 = ax.twiny()
    ax2.set_xlabel('durchschnitl. Calls / Wochentag '+title)
    ax2.set_xticks(np.arange(0,7,1))
    top_labels=dayf.av_angekommen.values
    ax2.set_xticklabels(top_labels)
    #ax2.set_xticklabels(['here','needs','to','be','the','average','value'])
    
    # Bars for each sum and day
    ax.bar(dayf.index,dayf.angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    ax.bar(dayf.index,dayf.verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
    ax.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

    ax2.bar(dayf.index,np.arange(0,7),width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    # Markers for Mondays and Months
    #plt.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize='4', linestyle='none', label='Montage')
    #plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

    # Legend and Layout formatting after the plotting has taken place
    ax.legend()
    fig.tight_layout()


In [1376]:
average_week=av_week(dayf)
print average_week.av_angekommen
plot_average_week(average_week,dataset_title)


0    50.5
1    44.5
2    45.0
3    62.6
4    48.8
5    24.0
6     3.5
Name: av_angekommen, dtype: float64
